# A Quick Start Tutorial

This tutorial offers a quick-start guide for using _BaySpec_ to fit a spectral model to gamma-ray data. It can be broadly divided into the following three sections:

- Data: The spectra from Fermi/GBM's NaI detector and BGO detector.
- Model: A simple cutoff power-law function.
- Fitting: Bayesian inference implemented using _multinest_ (or _emcee_).

In [1]:
import numpy as np
from bayspec.model.local import *
from bayspec import DataUnit, Data, Infer, Plot

In [2]:
savepath = 'quickstart'

1. Load spectra and response data.

In [3]:
nai = DataUnit(
    src='./ME/me.src', 
    bkg='./ME/me.bkg', 
    rsp='./ME/me.rsp', 
    notc=[8, 900], 
    stat='pgstat', 
    rebn={'min_sigma': 2, 'max_bin': 10})

bgo = DataUnit(
    src='./HE/he.src', 
    bkg='./HE/he.bkg', 
    rsp='./HE/he.rsp', 
    notc=[300, 38000], 
    stat='pgstat', 
    rebn={'min_sigma': 2, 'max_bin': 10})

data = Data([('nai', nai), 
             ('bgo', bgo)])

data.save(savepath)
print(data)

╒════════╤════════════════╤═════════════╤════════════╤════════╕
│  Name  │    Noticing    │  Statistic  │  Grouping  │  Time  │
╞════════╪════════════════╪═════════════╪════════════╪════════╡
│  nai   │   [[8, 900]]   │   pgstat    │    None    │  None  │
├────────┼────────────────┼─────────────┼────────────┼────────┤
│  bgo   │ [[300, 38000]] │   pgstat    │    None    │  None  │
╘════════╧════════════════╧═════════════╧════════════╧════════╛
╒════════╤═════════════╤═════════════╤═════════╤═════════╤══════════╕
│  par#  │  Component  │  Parameter  │  Value  │  Prior  │  Frozen  │
╞════════╪═════════════╪═════════════╪═════════╪═════════╪══════════╡
│   1    │     nai     │     sf      │    1    │  None   │   True   │
├────────┼─────────────┼─────────────┼─────────┼─────────┼──────────┤
│   2    │     nai     │     bf      │    1    │  None   │   True   │
├────────┼─────────────┼─────────────┼─────────┼─────────┼──────────┤
│   3    │     nai     │     rf      │    1    │  None   │   T

2.Define spectral model.

In [4]:
model = cpl()
model.save(savepath)
print(model)

cpl [add]
cutoff power law model
╒════════╤═════════════╤═════════════╤═════════╕
│  cfg#  │  Component  │  Parameter  │  Value  │
╞════════╪═════════════╪═════════════╪═════════╡
│   1    │     cpl     │  redshift   │    0    │
╘════════╧═════════════╧═════════════╧═════════╛
╒════════╤═════════════╤═════════════╤═════════╤═════════════╤══════════╕
│  par#  │  Component  │  Parameter  │  Value  │    Prior    │  Frozen  │
╞════════╪═════════════╪═════════════╪═════════╪═════════════╪══════════╡
│   1    │     cpl     │  $\alpha$   │   -1    │ unif(-8, 4) │  False   │
├────────┼─────────────┼─────────────┼─────────┼─────────────┼──────────┤
│   2    │     cpl     │ log$E_{c}$  │    2    │ unif(0, 4)  │  False   │
├────────┼─────────────┼─────────────┼─────────┼─────────────┼──────────┤
│   3    │     cpl     │   log$A$    │   -1    │ unif(-6, 5) │  False   │
╘════════╧═════════════╧═════════════╧═════════╧═════════════╧══════════╛



3.Run Bayesian inference.

In [5]:
infer = Infer([(data, model)])
infer.save(savepath)
print(infer)

╒════════╤═════════╤══════════════╤═════════════╤═════════════╤═════════╕
│  cfg#  │  Class  │  Expression  │  Component  │  Parameter  │  Value  │
╞════════╪═════════╪══════════════╪═════════════╪═════════════╪═════════╡
│   1    │  model  │     cpl      │     cpl     │  redshift   │    0    │
╘════════╧═════════╧══════════════╧═════════════╧═════════════╧═════════╛
╒════════╤═════════╤══════════════╤═════════════╤═════════════╤═════════╤═════════════╕
│  par#  │  Class  │  Expression  │  Component  │  Parameter  │  Value  │    Prior    │
╞════════╪═════════╪══════════════╪═════════════╪═════════════╪═════════╪═════════════╡
│   1*   │  model  │     cpl      │     cpl     │  $\alpha$   │   -1    │ unif(-8, 4) │
├────────┼─────────┼──────────────┼─────────────┼─────────────┼─────────┼─────────────┤
│   2*   │  model  │     cpl      │     cpl     │ log$E_{c}$  │    2    │ unif(0, 4)  │
├────────┼─────────┼──────────────┼─────────────┼─────────────┼─────────┼─────────────┤
│   3*   │  mo

In [6]:
post = infer.multinest(nlive=400, resume=True, verbose=False, savepath=savepath)
post.save(savepath)
print(post)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  400
 dimensionality =    3
 resuming from previous job
 *****************************************************
  analysing data from quickstart/1-.txt
 ln(ev)=  -296.28570556179801      +/-  0.20874724397948941     
 Total Likelihood Evaluations:        12469
 Sampling finished. Exiting MultiNest
╒════════╤═════════╤══════════════╤═════════════╤═════════════╤════════╤══════════╤════════╤══════════════════╕
│  par#  │  Class  │  Expression  │  Component  │  Parameter  │  Mean  │  Median  │  Best  │    1sigma CI     │
╞════════╪═════════╪══════════════╪═════════════╪═════════════╪════════╪══════════╪════════╪══════════════════╡
│   1    │  model  │     cpl      │     cpl     │  $\alpha$   │ -1.562 │  -1.562  │ -1.563 │ [-1.572, -1.552] │
├────────┼─────────┼──────────────┼─────────────┼─────────────┼────────┼──────────┼────────┼───────────

In [7]:
fig = Plot.infer(post, style='CE')
fig.save(f'{savepath}/ctsspec')

In [8]:
fig = Plot.infer(post, style='NE')
fig.save(f'{savepath}/phtspec')

In [9]:
fig = Plot.post_corner(post, ploter='plotly')
fig.save(f'{savepath}/corner')

In [10]:
earr = np.logspace(1, 3, 100)

modelplot = Plot.model(ploter='plotly', style='vFv', CI=True)
modelplot.add_model(model, E=earr)
fig = modelplot.get_fig()
fig.save(f'{savepath}/model')

In [11]:
ergflux = model.best_ergflux(emin=10, emax=1000, ngrid=1000)
ergflux_sample = model.ergflux_sample(emin=10, emax=1000, ngrid=1000)

print(ergflux)
print(ergflux_sample)

8.364073671737029e-06
{'mean': 8.370509833416337e-06, 'median': 8.370525717671357e-06, 'Isigma': array([8.31300014e-06, 8.42928731e-06]), 'IIsigma': array([8.25616620e-06, 8.48290234e-06]), 'IIIsigma': array([8.19526046e-06, 8.53830033e-06])}
